# 디스크립터 실전  
- 디스크립터를 통해 처리할 수 있는 몇 가지 상황 알아보기  
- 디스크립터에 대한 여러 구현 방법과 각각의 장단점 등 살펴보기
- 디스크립터를 사용하기에 가장 적합한 시나리오는 어떤 것들이 있는 지 알아보기  

## 디스크립터를 사용한 애플리케이션  
- 최종적으로 코드 중복을 최소화 하는 방향의 예제로 발전시키기 
- 중복 코드를 디스크립터로 추상화함으로써 클라이언트의 코드 줄이기

### 디스크립터를 사용하지 않은 예  
예제: 속성을 가진 일반 클래스의 속성 값이 달라질 때마다 추적  
-> 방법1: 속성의 setter 메서드에서 값이 변경될 때 검사하여 리스트와 같은 내부 변수에 값을 저장하기  
예제의 내용: 여행자 클래스, 현재 어느 도시에 있는 지를 속성으로 가짐. 사용자가 방문한 모든 도시를 추적

In [1]:
class Traveller:
    def __init__(self, name, current_city):
        self.name = name
        self._current_city = current_city
        self._cities_visited = [current_city]
        
    @property
    def current_city(self):
        return self._current_city
    
    @current_city.setter
    def current_city(self, new_city):
        if new_city != self._current_city:
            self._cities_visited.append(new_city)
            self._current_city = new_city
            
    @property
    def cities_visited(self):
        return self._cities_visited

In [4]:
alice = Traveller("Alice", "Barcelona")
alice.current_city = "Paris"
alice.current_city = "Brussels"
alice.current_city = "Amsterdam"
alice.cities_visited

['Barcelona', 'Paris', 'Brussels', 'Amsterdam']

위의 기능만 구현해도 된다면 추가로 구현할 필요 없이 프로퍼티만 사용해도 충분함.  
그러나 애플리케이션 여러 곳에서 똑같은 로직을 사용하여 속성의 모든 변수를 추적해야한다면 같은 로직을 반복해야함.  

### 이상적인 구현 방법  
모든 클래스에 적용할 수 있도록 디스크립터를 사용하여 이전 섹션의 문제를 해결해보기  
속성에 대해 이름을 가진 일반적인 디스크립터 생성. 이 디스크립터는 값이 달라질 경우 리스트에 저장하여 추적하는 기능을 가짐  

디스크립터는 특정 도메인과 관련이 없게 다른 유형의 클래스에서도 동일한 결과를 내도록 설계되었음

In [5]:
class HistoryTracedAttribute:
    def __init__(self, trace_attribute_name) -> None:
        """
        속성의 이름은 디스크립터에 할당된 변수 중 하나로 이 예제에서는 current_city임
        이에대한 추적을 저장할 변수의 이름을 디스크립터에 전달.
        이 예에서는 cities_visited라는 속성에 current_city의 모든 값을 추적하도록 지시
        """  
        self.trace_attribute_name = trace_attribute_name
        self._name = None
        
    def __set_name__(self, owner, name):
        self._name = name
        
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return instance.__dict__[self._name] 
    
    def __set__(self, instance, value):
        self._track_change_in_value_for_instance(instance, value)
        instance.__dict__[self._name] = value
        
    def _track_change_in_value_for_instance(self, instance, value):
        """
        디스크립터를 처음으로 호출할 때는 추적 값이 존재하지 않을 것이므로 나중에 추가할 수 있도록 비어있는 배열로 초기화
        """
        self._set_default(instance)
        if self._needs_to_track_change(instance, value):
            instance.__dict__[self.trace_attribute_name].append(value)
            
    def _needs_to_track_change(self, instance, value) -> bool:
        try:
            current_value = instance.__dict__[self._name]
        except KeyError:
            """
            처음 Traveller를 호출할 때는 방문지가 없으므로 인스턴스 사전에서 current_city 키도 존재하 않음
            이런 경우도 새로운 여행지가 생긴 것이므로 추적의 대상이 됨.
            """
            return true
        return value != current_value # 새 값이 설정된 값과 다를 때만 변경 사항 저장
    
    def _set_default(self, instance):
        instance.__dict__.setdefault(self.trace_attribute_name, []) 
        # 사전의 setdefault 메서드는 KeyError를 피하기 위해서 사용됨
        # setdefault는 두 개의 파라미터를 받는데 첫 번째 파라미터의 키가 있으면 해당 값을 반환하고, 없으면 두번째 파라미터를 반환
        
class Traveller:
    current_city = HistoryTracedAttribute("cities_visited")
    
    def __init__(self, name, current_city):
        """
        Traveller의 __init__메서드에서 디스크립터가 이미 생성된 단계.
        할당 명령은 2단계 값을 추적하기 위한 빈 리스트 만들기를 실행하고, 
        3단계를 실행하여 리스트에 값을 추가하고 나중에 검색하기 위한 키를 설정함
        """
        self.name = name
        self.current_city = current_city

디스크립터 코드는 다소 복잡해졌으나 클라이언트 클래스의 코드는 상당히 간단해짐.  
위의 디스크립터를 여러번 사용한다면 디스크립터를 사용하는 것이 클래스마다 중복 코드를 생성하는 것보다 나음  
디스크립터 안에는 어떠한 비즈니스 로직도 포함되어 있지 않기 때문에 완전히 다른 어떤 클래스에서 적용하여도 같은 효과를 냄  
디스크립터는 비즈니스 로직의 구현보다는 라이브러리, 프레임워크 또는 내부 API를 정의하는 데 적합함.

## 다른 형태의 디스크립터
### 전역 상태 공유 이슈
디스크립터는 클래스 속성으로 설정해야함. 
- 클래스 속성의 문제점: 해당 클래스의 모든 인스턴스에서 공유됨
- 디스크립터 객체에 데이터를 보관하면 모든 객체가 동일한 값에 접근할 수 있음

아래의 코드는 각 객체에 데이터를 저장하는 대신 디스크립터가 데이터 자체를 유지하도록 잘못 정의하면 어떻게 되는 지에 대한 예시

In [9]:
class SharedDataDescriptor:
    def __init__(self, initial_value):
        self.value = initial_value
        
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return self.value
    
    def __set__(self, instance, value):
        self.value = value
        
class ClientClass:
    descriptor = SharedDataDescriptor("첫 번째 값")

In [10]:
client1 = ClientClass()
print("1. "+client1.descriptor)

client2 = ClientClass()
print("2. "+client2.descriptor)
client2.descriptor = "client2를 위한 값"
print("3. "+client2.descriptor)
print("4. "+client1.descriptor)

1. 첫 번째 값
2. 첫 번째 값
3. client2를 위한 값
4. client2를 위한 값


한 객체의 값을 변경하면 갑자기 모든 객체의 값이 한꺼번에 변경됨  
이는 ClientClass.descriptor가 고유하기 때문  

이를 해결하기 위해서 디스크립터는 각 인스턴스의 값을 보관했다가 반환해야 함.  
이것이 각 인스턴스의 &#95;&#95;dict&#95;&#95; 사전에 값을 설정하고 검색하는 이유

앞에서 살펴본 바에 따르면 getattr()과 setattr()을 사용할 수 없으므로 &#95;&#95;dict&#95;&#95; 속성을 수정하는 것이 최후의 사용 가능한 선택임

### 객체의 사전에 접근하기
디스크립터는 객체의 사전 &#95;&#95;dict&#95;&#95;에 값을 저장하고 조회함

### 약한 참조 사용
&#95;&#95;dict&#95;&#95;를 사용하지 않으려는 경우 또 다른 대안은 디스크립터 객체가 직접 내부 매핑을 통해 각 인스턴스의 값을 보관하고 반환하는 것  
단, 내부 매핑을 할 때 사전을 사용하지 않도록 주의할 것  
- 클라이언트 클래스는 디스크립터에 대한 참조를 가지며 디스크립터는 디스크립터를 사용하는 객체에 대한 참조를 가지므로 순환 종속성이 생겨 결과적으로 결코 가비지 컬렉션이 되지 않는 문제가 있음  
- 이를 해결하기 위해 사전은 weakref(WEAKREF 01) 모듈에 정의된 것처럼 약한 키가 되어야 함 (약한 참조만으로는 객체를 살아있도록 유지할 수 없음)

이 경우 디스크립터의 코드는 다음과 같음

In [13]:
from weakref import WeakKeyDictionary

class DescriptorClass:
    def __init__(self, initial_value):
        self.value = initial_value
        self.mapping = weakKeyDictionary()
        
    def __get__(self, instacne, owner):
        if instance is None:
            return self
        return self.mapping.get(instance, self.value)
    
    def __set__(self, instance, value):
        self.mapping[instance] = value

위와 같이 하면 문제가 해결되나 몇 가지 고려사항이 있음  
- 인스턴스 객체는 더이상 속성을 보유하지 않고 대신 디스크립터가 속성을 보유함  
    - 객체의 사전에 있는 내용을 찾으려고 해도 (e.g. vars(client) 호출) 객체는 속성을 보유하지 않으므로 완전한 데이터를 반환하지 않음  
- 객체는 &#95;&#95;hash&#95;&#95; 메서드를 구현하여 해시가 가능해야함. 만약 해시가 가능하지 않으면 WeakKeyDictionary에 매핑할 수 없음  

이러한 이유로 각 인스턴스의 &#95;&#95;dict&#95;&#95; 사전을 사용하는 방법을 선호 

## 디스크립터에 대한 추가 고려사항  
디스크립터를 사용하는 것이 좋은 선택일 경우 어떤 일을 할 수 있는 지, 처음에 접근했던 방법을 디스크립터를 사용해 어떻게 개선할 수 있을 지에 대한 관점에서 고려사항 검토하기  
디스크립터를 사용해 구현한 후에 원래 구현과 비교하여 장단점 비교해보기  

### 코드 재사용  
디스크립터는 코드 중복을 피하기 위한 일반적인 도구이자 강력한 추상화 도구임  
디스크립터가 필요한 곳을 찾는 가장 좋은 방법은 프로퍼티가 필요한 구조가 반복되는 경우를 찾는 것  

**프로퍼티**: 디스크립터의 특수한 경우 (@property 데코레이터는 get, set, delete 작업을 정의하여 디스크립터 프로토콜을 모두 구현한 디스크립터  
-> 디스크립터는 프로퍼티보다 훨씬 복잡한 작업에 사용될 수 있음  

디스크립터는 데코레이터가 클래스 메서드에서도 동작할 수 있도록 도와 더 나은 데코레이터를 만들 수 있게 함  
앞서 데코레이터를 사용할 만한 가치가 있는 지 결정하기 위해 활용한 3의 규칙을 디스크립터 사용 가치를 판별할 때도 활용할 수 있음  

3의 규칙 외에도 클라이언트가 사용하게 되는 내부 API에 대해서는 디스크립터를 사용하는 것이 좋음  
-> 일회성 솔루션이 아닌 라이브러리나 프레임워크의 디자인에 대해서는 기능을 확장하기 좋기 때문  

- 디스크립터에는 특별한 이유나 확실한 장점이 없는 이상 비즈니스 로직을 넣으면 안됨  
- 디스크립터의 코드에는 비즈니스 코드가 아닌 구현 코드가 더 많이 포함되어야 함  
- 디스크립터는 비즈니스 로직에서 사용할 새로운 객체나 데이터 구조를 정의하는 것과 유사

### 클래스 데코레이터 피하기
5장에서 이벤트 객체의 직렬화 방식을 결정하기 위해 사용한 클래스 데코레이터를 떠올려 보면 두 개의 클래스 데코레이터를 사용하여 구현하였음
```python
@Serialization(
    username=show_original,
    password=hide_field, 
    ip=show_original, 
    timestamp=format_time
)
@dataclass
class LoginEvent:
    username: str
    password: str
    ip: str
    timestamp: datetime
```

첫 번째 것은 어노테이션에서 속성을 가져와 변수를 선언할 때 사용하는 반면 두번째는 파일을 처리하는 방법을 정의함  
위의 두 가지 데코레이터를 디스크립터로 변경하는 것에 대한 예제  
-> 각 속성의 값에 대해 요건에 맞게 변환 후 수정된 벼전을 반환하는 디스크립터를 만듦

In [16]:
from functools import partial
from typing import Callable

class BaseFieldTransformation:
    
    def __init__(self, transformation: Callable[[], str]) -> None:
        self._name = None
        self.transformation = transformation
        
    def __get__(self, instance, owner):
        if instance is None:
            return self
        raw_value = instance.__dict__[self._name]
        return self.transformation(raw_value)
    
    def __set_name__(self, owner, name):
        self._name = name
        
    def __set__(self, instance, value):
        instance.__dict__[self._name] = value
        
ShowOriginal = partial(BaseFieldTransformation, transformation=lambda x: x)
HideField = partial(BaseFieldTransformation, transformation=lambda x: "**민감한 정보 삭제**")
FormatTime = partial(BaseFieldTransformation, transformation=lambda ft: ft.strftime("%Y-%m-%d %H:%M"))

하나의 파라미터를 취하여 하나의 값을 반환하는 함수를 가진 디스크립터  
위 예제는 하위 클래스를 추가 생성하는 방법을오 functools.partial을 사용  
- functools.partial()은 하나 이상의 인수가 이미 채워진 새 버전의 함수를 만들 때 사용하는 함수  

클래스 변환 함수에 호출 가능한 함수를 직접 전달하여 함수의 새 버전을 생성  

이벤트 클래스는 아래와 같음

In [17]:
class LoginEvent:
    username = ShowOriginal()
    password = HideField()
    ip = ShowOriginal()
    timestamp = FormatTime()
    
    def __init__(self, username, password, ip, timestamp):
        self.username = username
        self.password = password
        self.ip = ip
        self.timestamp = timestamp
        
    def serialize(self):
        return {
            "username": self.username,
            "password": self.password,
            "ip": self.ip,
            "timestamp": self.timestamp
        }

In [20]:
from datetime import datetime

le = LoginEvent("john", "secret password", "1.1.1.1", datetime.utcnow())
print(vars(le))
print(le.serialize())
print(le.password)

{'username': 'john', 'password': 'secret password', 'ip': '1.1.1.1', 'timestamp': datetime.datetime(2022, 8, 6, 6, 52, 46, 311276)}
{'username': 'john', 'password': '**민감한 정보 삭제**', 'ip': '1.1.1.1', 'timestamp': '2022-08-06 06:52'}
**민감한 정보 삭제**


데코레이터를 사용한 이전 구현과 비교하였을 때의 차이점
- serialize() 메서드를 추가하고 필드를 결과 사전에 표시하기 전에 숨겼으나 메모리의 이벤트 인스턴스에서 변환을 적용하지 않은 원래의 값을 구할 수 있음  
- 객체의 public 속성을 요청하면 디스크립터가 결과를 보여주기 전에 변환 작업을 적용함. 객체의 &#95;&#95;dict&#95;&#95; 사전에 접근하여 원본 값을 가져올 수 있음. 그러나 기본적으로는 값을 요청하면 변환된 값을 반환함  
- 템플릿 메서드 디자인 패턴으로 클래스마다 고유한 변환 함수를 갖도록 클래스 계층 구조를 만들 수 있으나 이번 예제에서는 파생 클래스의 변경 사항이 상대적으로 적기 때문에 (1개의 함수) 파생 클래스를 기본 클래스의 부분 애플리케이션으로 만드는 방법을 선택함  
- 새로운 변환 필드를 생성하는 것은 새로운 기본 클래스를 정의하는 것만큼 간단해야함.  
- 디스크립터는 객체이므로 모델을 만들어서 객체지향 프로그래밍의 규칙을 적용 가능  
    - 위의 예제는 4장 SOLID 원칙의 OCP를 따름
    - 새로운 변환 기능을 추가할 때 기본 클래스를 수정하지 않고 파생 클래스를 만들면 되기 때문
    
&#95;&#95;init&#95;&#95;()과 serialize() 메서드를 구현한 기본 클래스를 만들고 그것을 상속받아 LoginEvent 클래스를 간단히 정의하는 방법은 아래와 같음
```python
class LoginEvent(BaseEvent):
    username = ShowOriginal()
    password = HideField()
    ip = ShowOriginal()
    timestamp = FormatTime()
```
이렇게 코드를 작성하면 클래스가 훨씬 깔끔해 보임.  
기본 클래스는 공통 메서드만 추상화 할 것이고 결과적으로 각 이벤트 클래스는 더 작고 간단하게 됨  
각 이벤트 클래스가 단순해질 뿐 아니라 디스크립터 자체도 매우 작아서 클래스 데코레이터보다 훨씬 간단하므로 디스크립터를 사용한 방식이 보다 뛰어남